# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission, if necessary. Sections that begin with **'Implementation'** in the header indicate where you should begin your implementation for your project. Note that some sections of implementation are optional, and will be marked with **'Optional'** in the header.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---

## Step 1: Dataset Exploration

Visualize the German Traffic Signs Dataset. This is open ended, some suggestions include: plotting traffic signs images, plotting the count of each sign, etc. Be creative!


The pickled data is a dictionary with 4 key/value pairs:

- features -> the images pixel values, (width, height, channels)
- labels -> the label of the traffic sign
- sizes -> the original width and height of the image, (width, height)
- coords -> coordinates of a bounding box around the sign in the image, (x1, y1, x2, y2). Based the original image (not the resized version).

In [ ]:
# Let's first load all the modules we're going to need

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import tensorflow as tf
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tqdm import tqdm_notebook

%matplotlib inline

In [ ]:
# Load pickled data

# TODO: fill this in based on where you saved the training and testing data
training_file = 'data/train.p'
testing_file = 'data/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [ ]:
### To start off let's do a basic data summary.

# TODO: number of training examples
n_train = len(X_train)

# TODO: number of testing examples
n_test = len(X_test)

# TODO: what's the shape of an image?
image_shape = X_train[0].shape

# TODO: how many classes are in the dataset
# y_train_unique_indices contains the index of the first occurrence of each class in y_train
# We'll need this in order to plot a sample image for each label
y_train_unique, y_train_unique_indices = np.unique(y_train, return_index=True)
n_classes = len(y_train_unique)

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

In [1]:
### Data exploration visualization goes here.
### Feel free to use as many code cells as needed.

# Load the label names (useful for plotting, and later for displaying predictions)
label_names = np.genfromtxt('signnames.csv', delimiter=',', dtype="i8,S256", skip_header=1)

In [ ]:
# Getting the count of each label in both training and test data
y_train_bincount = np.bincount(y_train)
y_test_bincount = np.bincount(y_test)
y_train_binperc = np.divide(y_train_bincount, n_train)
y_test_binperc = np.divide(y_test_bincount, n_test)

assert len(y_train_bincount) == len(y_test_bincount),\
    'Training and test label sets should have the same number of labels'

print('Showing label distribution in training and test data:')
for i in range(len(y_train_bincount)):
    print('  Label {0:2d} {1:53s} - Training: {2:4d} ({3:5.2f}%) Test: {4:4d} ({5:5.2f}%)'
          .format(i, label_names[i][1], y_train_bincount[i], 100 * y_train_binperc[i], y_test_bincount[i], 100 * y_test_binperc[i]))

# Showing the label distribution in the training and test samples
%matplotlib inline
plot = plt.subplot(111)
plot.set_title('Label Occurrences (%)')
plot.plot(y_train_binperc * 100, 'r', label='Training')
plot.plot(y_test_binperc * 100, 'b', label='Test')
plot.set_xlim([-1, n_classes])
plot.legend()
plt.show()

In [ ]:
# Show the first occurrence of each label in the data set
plt.rcParams["figure.figsize"] = [12, 36]
n_img_to_show = n_classes
num_cols = 4
num_rows = np.ceil(n_img_to_show / num_cols)
k = 1
print('Showing a sample of', n_img_to_show, 'images:')
for i in y_train_unique_indices:
    image = X_train[i]
    image_label_index = y_train[i]
    image_label_text = label_names[image_label_index][1]
    subplot = plt.subplot(num_rows, num_cols, k)
    subplot.imshow(image)
    subplot.set_title(image_label_text)
    k = k + 1

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

There are various aspects to consider when thinking about this problem:

- Your model can be derived from a deep feedforward net or a deep convolutional network.
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [1]:
### Preprocess the data here.
### Feel free to use as many code cells as needed.

# Convert to grayscale, normalize, and flatten to 1D
alpha = -0.5
beta = 0.5

def preprocess_image(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_norm = cv2.normalize(gray_image, alpha, beta, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    gray_flat = np.array(gray_norm, dtype=np.float32).flatten()
    
    return gray_flat

def preprocess(array):
    normalized = []
    
    for i in range(len(array)):
        normalized.append(preprocess_image(array[i]))
                    
    return np.array(normalized)

In [ ]:
# Preprocess the features

X_train_gray_norm = preprocess(X_train)
X_test_gray_norm = preprocess(X_test)

print('Normalized images from a shape of', X_train[0].shape, 'to a shape of', X_train_gray_norm[0].shape)

# One-hot encode the labels and convert to float32

encoder = LabelBinarizer()
encoder.fit(y_train)
y_train_hot = encoder.transform(y_train)
y_test_hot = encoder.transform(y_test)
y_train_hot = y_train_hot.astype(np.float32)
y_test_hot = y_test_hot.astype(np.float32)

print('One-hot encoded labels from', y_train[0].shape, 'to a shape of', y_train_hot[0].shape)

In [ ]:
# For our previous small sample, let's plot the processed images against the originals to make
# sure all went well
plt.rcParams["figure.figsize"] = [12, 150]
num_cols = 2
num_rows = np.ceil((n_img_to_show * 2) / num_cols)
k = 1
print('Displaying the originals and the preprocessed images for the previous sample of', n_img_to_show, 'images:')
for i in y_train_unique_indices:
    image = X_train[i]
    image_gray_norm = X_train_gray_norm[i]
    
    image_label_index = y_train[i]
    image_label_text = label_names[image_label_index][1]
    
    subplot = plt.subplot(num_rows, num_cols, k)
    subplot.imshow(image)
    subplot.set_title(image_label_text)
    
    subplot = plt.subplot(num_rows, num_cols, k + 1)
    im = np.reshape(image_gray_norm, (image.shape[0], image.shape[1]))
    subplot.imshow(im, cmap='gray')
    subplot.set_title(image_label_text)
    
    k = k + 2

### Question 1 

_Describe the techniques used to preprocess the data._

**Answer:**

* Converted each feature image into grayscale
* Normalized the gray image to the [-0.5, 0.5] range
* Flattened the gray normalized image into a 1-dimensional vector
* One-hot encoded the labels (converted each label from an integer value into a 1-dimensional binary vector that only has the corresponding bit turned on).

This was applied to both the training and the test samples.

In [2]:
### Generate data additional (if you want to!)
### and split the data into training/validation/testing sets here.
### Feel free to use as many code cells as needed.

# Not additional data generated. See answer below.

# Splitting the data

features_training = X_train_gray_norm
labels_training = y_train_hot
features_test = X_test_gray_norm
labels_test = y_test_hot

num_features = features_training[0].shape[0]
num_labels = labels_training[0].shape[0]

assert features_training[0].shape == features_test[0].shape,\
    'Training and test feature sets must have the same shape'
assert labels_training[0].shape == labels_test[0].shape,\
    'Training and test label sets must have the same shape'
assert len(features_training) == len(labels_training),\
    'Training features and labels must have the same size'
assert len(features_test) == len(labels_test),\
    'Test features and labels must have the same size'

total_training = len(features_training)
total_test = len(features_test)
total = len(features_training) + len(features_test)
print('Preparing the data for training:')
print('\tTotal:', total)
print('\t{0:6d} ({1:5.2f}%) training examples'\
      .format(total_training, 100. * total_training / total))
print('\t{0:6d} ({1:5.2f}%) test examples'\
      .format(total_test, 100. * total_test / total))
print('Shape of features is', features_training[0].shape, ', and labels is', labels_training[0].shape)

validation_ratio = 0.05
random_state = 20161128
features_training, features_validation, labels_training, labels_validation = train_test_split(
    features_training,
    labels_training,
    test_size=validation_ratio,
    random_state=random_state)

print('Extracted {0:.2f}% from the training sample to act as validation'.format(validation_ratio * 100))

total_training = len(features_training)
total_validation = len(features_validation)
total_test = len(features_test)
total = len(features_training) + len(features_validation) + len(features_test)
print('\tTotal:', total)
print('\t{0:6d} ({1:5.2f}%) training examples'\
      .format(total_training, 100. * total_training / total))
print('\t{0:6d} ({1:5.2f}%) validation examples'\
      .format(total_validation, 100. * total_validation / total))
print('\t{0:6d} ({1:5.2f}%) test examples'\
      .format(total_test, 100. * total_test / total))

### Question 2

_Describe how you set up the training, validation and testing data for your model. If you generated additional data, why?_

**Answer:**

I extracted 5% from the training examples to act as a validation set.

***Additionnal data***

I did not generate additional data here. But it would be useful for the model to include some fake data that will cover cases encountered when running the model in real conditions. These conditions are not accounted for by the current data, and including them in the training will make the model much more robust in production.

For example:
* Resolution change: by blurring or sharpening images over a reasonble range. This will cover focus issues in real conditions.
* Distortion around an axis that goes through the center of the image (no need to do it for an axis that does not go through the center since we can always re-center the sign during preprocessing). This is useful for cases when the sign does not face the camera properly.
* Rotation around the center, over a reasonable range. This will take account of signs that are slightly bent or turned. And for windy conditions.
* Saturation and noise, to cover bright and low light conditions.
* Adding artefacts to account for rain. snow, sand, fog, smoke, etc.
* Occlude parts of the image, since real conditions can have a vehicle, a tree, a building, etc, obstruct parts of the sign.
* Color change to account for sign discoloration.
* Extra artefacts to account for vandalized or old signs (graffitis for example).

It's possible to use OpenCV to generate such data, with tweakable ratios of how much each condition contributes to the total. Obviously, the difficulty in generating such data varies by condition, so it'd be useful to start by the cases that are easiest to generate and evaluate their impact during the training. Before investing much effort in covering all the extra conditions.

In [6]:
### Define your architecture here.
### Feel free to use as many code cells as needed

# This is a simple linear model I used for testing the pipeline (inspired from the Tensorflow Lab)
def linearModel(num_features, num_labels):
    x = tf.placeholder(tf.float32)
    y = tf.placeholder(tf.float32)

    weights = tf.Variable(tf.truncated_normal((num_features, num_labels)))
    biases = tf.Variable(tf.zeros(num_labels))
    
    logits = tf.matmul(x, weights) + biases
    
    return x, y, logits

# This is the perceptron model from the course
def perceptron(n_input, n_classes):
    n_hidden_layer = 256

    weights = {
        'hidden_layer': tf.Variable(tf.random_normal([n_input, n_hidden_layer])),
        'out': tf.Variable(tf.random_normal([n_hidden_layer, n_classes]))
    }
    biases = {
        'hidden_layer': tf.Variable(tf.random_normal([n_hidden_layer])),
        'out': tf.Variable(tf.random_normal([n_classes]))
    }
    
    x = tf.placeholder(tf.float32)
    y = tf.placeholder(tf.float32)

    # Hidden layer with RELU activation
    layer_1 = tf.nn.relu(tf.matmul(x, weights['hidden_layer']) + biases['hidden_layer'])
    
    # Output layer with linear activation
    logits = tf.matmul(layer_1, weights['out']) + biases['out']
    
    return x, y, logits

# ConvNet
def convnet(n_input, n_classes):
    x = tf.placeholder(tf.float32, [None, n_input])
    y = tf.placeholder(tf.float32, [None, n_classes])
    
    # Create some wrappers for simplicity
    def conv2d(x, W, b, strides=1):
        # Conv2D wrapper, with bias and relu activation
        x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
        x = tf.nn.bias_add(x, b)
        return tf.nn.relu(x)

    def maxpool2d(x, k=2):
        # MaxPool2D wrapper
        return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

    # Create model
    def conv_net(x, weights, biases, dropout_conv, dropout):
        # Reshape input picture
        x = tf.reshape(x, shape=[-1, 32, 32, 1])# Modified here

        # Convolution Layer
        conv1 = conv2d(x, weights['wc1'], biases['bc1'])
        # Max Pooling (down-sampling)
        conv1 = maxpool2d(conv1, k=2)
        conv1 = tf.nn.dropout(conv1, dropout_conv)

        # Convolution Layer
        conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
        # Max Pooling (down-sampling)
        conv2 = maxpool2d(conv2, k=2)
        conv2 = tf.nn.dropout(conv2, dropout_conv)
        
        # Convolution Layer
        conv3 = conv2d(conv2, weights['wc3'], biases['bc3'])
        # Max Pooling (down-sampling)
        conv3 = maxpool2d(conv3, k=2)
        conv3 = tf.nn.dropout(conv3, dropout_conv)

        # Fully connected layer
        # Reshape conv2 output to fit fully connected layer input
        fc1 = tf.reshape(conv3, [-1, weights['wd1'].get_shape().as_list()[0]])
        fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
        fc1 = tf.nn.relu(fc1)
        # Apply Dropout
        fc1 = tf.nn.dropout(fc1, dropout)
        
        # Fully connected layer
        # Reshape conv2 output to fit fully connected layer input
        #fc2 = tf.reshape(fc1, [-1, weights['wd2'].get_shape().as_list()[0]])
        fc2 = tf.add(tf.matmul(fc1, weights['wd2']), biases['bd2'])
        fc2 = tf.nn.relu(fc2)
        # Apply Dropout
        fc2 = tf.nn.dropout(fc2, dropout)
        
        # Output, class prediction
        out = tf.add(tf.matmul(fc2, weights['out']), biases['out'])
        return out

    # Store layers weight & bias
    weights = {
        # 3x3 conv, 1 input, 32 outputs
        'wc1': tf.Variable(tf.truncated_normal([3, 3, 1, 32])),
        # 3x3 conv, 32 inputs, 64 outputs
        'wc2': tf.Variable(tf.truncated_normal([3, 3, 32, 64])),
        # 3x3 conv, 64 inputs, 128 outputs
        'wc3': tf.Variable(tf.truncated_normal([3, 3, 64, 128])),
        # fully connected, 4*4*128 inputs, 256 outputs
        'wd1': tf.Variable(tf.truncated_normal([4*4*128, 256])),
        # fully connected, 256 inputs, 512 outputs
        'wd2': tf.Variable(tf.truncated_normal([256, 512])),
        # 512 inputs, n_classes outputs (class prediction)
        'out': tf.Variable(tf.truncated_normal([512, n_classes]))
    }

    biases = {
        'bc1': tf.Variable(tf.zeros([32])),
        'bc2': tf.Variable(tf.zeros([64])),
        'bc3': tf.Variable(tf.zeros([128])),
        'bd1': tf.Variable(tf.zeros([256])),
        'bd2': tf.Variable(tf.zeros([512])),
        'out': tf.Variable(tf.zeros([n_classes]))
    }

    # Construct model
    logits = conv_net(x, weights, biases, keep_prob_conv, keep_prob)
    
    return x, y, logits

class Model:
    def __init__(self, name, function, path):
        self.name = name
        self.function = function
        self.path = path

#model_to_use = Model('Linear', linearModel, 'models/linear.cpkt')
#model_to_use = Model('Percepetron', perceptron, 'models/percepetron.cpkt')
model_to_use = Model('ConvNet', convnet, 'models/convnet.cpkt')

keep_prob_conv = tf.placeholder(tf.float32) #dropout (keep probability for convolutional layers)
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability for fully-connected layers)

features, labels, logits = model_to_use.function(num_features, num_labels)

prediction = tf.nn.softmax(logits)
#cross_entropy = 
#cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(prediction), reduction_indices=1))
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, labels))

# Determine if the predictions are correct
is_correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(labels, 1))
#correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))

# Accurracy of the predictions
accuracy = tf.reduce_mean(tf.cast(is_correct_prediction, tf.float32))

print('Created the model to be used: {0}'.format(model_to_use.name))

### Question 3

_What does your final architecture look like? (Type of model, layers, sizes, connectivity, etc.)  For reference on how to build a deep neural network using TensorFlow, see [Deep Neural Network in TensorFlow
](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/b516a270-8600-4f93-a0a3-20dfeabe5da6/concepts/83a3a2a2-a9bd-4b7b-95b0-eb924ab14432) from the classroom._


**Answer:**

_The code of the models I experimented with is often inspired from the code examples given in the classroom._

I experimented with different architectures (see below for details), but the final model is a CNN with three 3x3 convolutional layers with RELU activation, each followed by 2x2 max pooling. The final two layers are fully connected layers with RELU activation of a size of 256 and 512 respectively.

I tried to add high dropout after each max pooling, and average dropout after the fully connected layers. This last step was meant to avoid the model from overfitting (which happens very quickly without dropout). See below for details.

In [5]:
### Train your model here.
### Feel free to use as many code cells as needed.

# The training loop here, with batching and final accurracy visualization is inspired from the
# Tensorflow Lab code.

# For linear
#epochs = 20
#batch_size = 1000
#slearning_rate = 0.1

# For perceptron
#epochs = 2000
#batch_size = 1000
#learning_rate = 0.1

# For convnet
epochs = 1000
batch_size = 1024
learning_rate = 0.0001
dropout_conv = 1.
dropout = 1.

# Measurements use for graphing cost and accuracy
log_batch_step = 50
batches = []
cost_batch = []
training_acc_batch = []
validation_acc_batch = []

# Feed dicts for training, validation, and test
training_feed_dict = {features: features_training, labels: labels_training, keep_prob_conv: dropout_conv, keep_prob: dropout}
validation_feed_dict = {features: features_validation, labels: labels_validation, keep_prob_conv: 1, keep_prob: 1}
test_feed_dict = {features: features_test, labels: labels_test, keep_prob_conv: 1, keep_prob: 1}

print("Training model '{0}'...".format(model_to_use.name))
print('epochs={0} batch_size={1} learning_rate={2}'.format(epochs, batch_size, learning_rate))

#optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

with tf.Session() as session:
    session.run(tf.initialize_all_variables())
    batch_count = int(np.ceil(len(features_training)/batch_size))
    
    for epoch_i in tqdm_notebook(range(epochs), desc='Epochs'):
        for batch_i in tqdm_notebook(range(batch_count), desc='Batches', leave=False):
            # Get a batch of training features and labels
            batch_start = batch_i * batch_size
            
            batch_features = features_training[batch_start:batch_start + batch_size]
            batch_labels = labels_training[batch_start:batch_start + batch_size]
            
            batch_feed_dict = {features: batch_features, labels: batch_labels, keep_prob_conv: dropout_conv, keep_prob: dropout}

            # Run optimizer and get cost
            _, current_cost = session.run([optimizer, cost], feed_dict=batch_feed_dict)

            # Log every 50 batches
            if not batch_i % log_batch_step:
                training_accuracy = session.run(accuracy, feed_dict=batch_feed_dict) #session.run(accuracy, feed_dict=training_feed_dict)
                validation_accuracy = session.run(accuracy, feed_dict=validation_feed_dict)

                previous_batch = batches[-1] if batches else 0
                batches.append(epoch_i * batch_count + batch_i)
                cost_batch.append(current_cost)
                training_acc_batch.append(training_accuracy * 100)
                validation_acc_batch.append(validation_accuracy * 100)
                
                print("Epoch={0:4d} Batch={1:3d} Cost={2:13.6f} Accuracy={3:5.2f}% Validation={4:5.2f}%".
                      format(epoch_i, batch_i, current_cost, training_acc_batch[-1], validation_acc_batch[-1]))

    # Compute the final costs and accurracies
    training_cost = cost_batch[-1]#session.run(cost, feed_dict=training_feed_dict)
    validation_cost = session.run(cost, feed_dict=validation_feed_dict)
    test_cost = session.run(cost, feed_dict=test_feed_dict)
    
    training_accuracy = training_acc_batch[-1]/100 #session.run(accuracy, feed_dict=training_feed_dict)
    validation_accuracy = session.run(accuracy, feed_dict=validation_feed_dict)
    test_accuracy = session.run(accuracy, feed_dict=test_feed_dict)
    
    # Persisting the model so that we can re-use it later
    saver = tf.train.Saver()
    save_path = saver.save(session, model_to_use.path)
    print('Final accurracies and costs:')
    print('  Training  : {0:5.2f}%, {1}'.format(training_accuracy * 100, training_cost))
    print('  Validation: {0:5.2f}%, {1}'.format(validation_accuracy * 100, validation_cost))
    print('  Test      : {0:5.2f}%, {1}'.format(test_accuracy * 100, test_cost))
    print("Trained model '{0}' saved to '{1}".format(model_to_use.name, save_path))

In [ ]:
# Plot cost along with training and validation accurraccies

%matplotlib inline

# We plot the cost with a log scale to better resolve the smallest values
cost_plot = plt.subplot(211)
cost_plot.plot(batches, cost_batch, 'g')
cost_plot.set_xlim([batches[0], batches[-1]])
cost_plot.set_yscale('log')
cost_plot.set_title('Cost')

acc_plot = plt.subplot(212)
acc_plot.plot(batches, training_acc_batch, 'r', label='Training')
acc_plot.plot(batches, validation_acc_batch, 'b', label='Validation')
acc_plot.set_xlim([batches[0], batches[-1]])
acc_plot.set_ylim([0, 100])
acc_plot.legend(loc=2)
acc_plot.set_title('Accuracy (%)')

plt.tight_layout()

plt.show()

### Question 4

_How did you train your model? (Type of optimizer, batch size, epochs, hyperparameters, etc.)_


**Answer:**

_The code of the loop I used for training the pipeline is inspired from the Tensorflow Lab code._

I batched the training data in sets of 1024 pairs for 1000 epochs (~37000 iterations). I used an AdamOptimizer for the optimization step. I printed the batch training accurracy and cost, along with the validation accuracy every 50 steps to monitor the convergence of the algorithm. And finally printed the accuracy on the test dataset, plotted the progress of cost and accurracies with the optimization.

### Question 5


_What approach did you take in coming up with a solution to this problem?_

**Answer:**

At the beginning, I tried to run a simple one hidden-layer neural network, based on the lessons and on the examples therein, such as the multi-layer perceptron. However, I encountered issues due to wrong normalization, incorrect vector sizes and unfamiliarity with TensorFlow and this type of optimization. These made the adaptation of such a model to my case challenging.

I therefore decided to go back to the simple linear model provided in the Tensorflow Lab and try to adapt my current pipeline to provide the right input to the model. Once this worked, and had a reasonable training output (80% accuracy on the test set after 100 epochs and a learning rate of 0.1 for a batch size of 128), I decided to go back to a non-linear model. Interestingly, this model gave quite good results with the external (web) dataset.

I returned to the multi-layer perceptron, and with insight gained from the accumulated experience thus far, I was able to apply the necessary changes to make it work. It converged very quickly and had a high accuracy on the training set (> 90%), a little lower on the validation set, and around 70% on the test set. This showed that it was likely overfitting the data. And not surprisingly, the results on the external images were less good than with the linear model.

I was finally ready to try a CNN architecture, similar to the one provided in the course. I attempted to run the code examples and other TensorFlow tutorials, but I was having very little success with them due to underfitting. So I modified the tutorial model slightly by adding one convolutional layer and one fully connected layer. This resulted in a much more reasonable accuracy. I then added dropout to avoid overfitting (the model without dropout reached 100% accurracy on the training set after 35 epochs with a batch size of 128 and a learning rate of 0.001), but the addition of the dropout seemed to underfit the data and brought no convergence. I increased the batch size help the model converge and decreased the learning rate to 0.0001.

In conclusion, I chose a simple model that illustrates the method of building deep neural architectures. I tried not to focus on the size of the network, as long as the resulting accuracy was acceptable, and the training time reasonable. Afterwards, it'd be interesting to explore how this model scales with a lot more data, and evaluate what is required to extend it in order to adapt it to those cases. This will also require to train it on a more capable machine.

---

## Step 3: Test a Model on New Images

Take several pictures of traffic signs that you find on the web or around you (at least five), and run them through your classifier on your computer to produce example results. The classifier might not recognize some local signs but it could prove interesting nonetheless.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [3]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

# For each image, load, crop to the proper aspect ratio, resize, convert to grayscale and normalize.
# Then, calculate the model's label prediction for it.

# TODO Don't assume the aspect ratio to be square. Use image_shape as input to determine how to crop the candidates
#      to have the same aspect ratio as the training set

candidates = []
candidates_names = []

print("Loading, optionally cropping, resizing to {0} and finally preprocessing extra images:\n".format(image_shape))

images_dir = "data/extra_images"
for image_name in os.listdir(images_dir):
    image_path = os.path.join(images_dir, image_name)
    if os.path.isdir(image_path):
        continue
    
    image = mpimg.imread(image_path)
    name, ext = os.path.splitext(image_name)
    print("  Handling '{0:37s}' of size {1}".format(image_path, image.shape))
    
    # If the image is rectangular, crop to a square around the center, by keeping the smallest length, so that
    # we have an identical aspect ratio as the training data
    if image.shape[0] != image.shape[1]:
        min_size = min(image.shape[0], image.shape[1])
        y_start = int(np.floor((image.shape[0] - min_size) / 2))
        y_end = int(np.floor((image.shape[0] + min_size) / 2))
        x_start = int(np.floor((image.shape[1] - min_size) / 2))
        x_end = int(np.floor((image.shape[1] + min_size) / 2))
        image = image[y_start:y_end, x_start:x_end]
        print('    Cropped to', image.shape)
    
    # Resize to our what our model expects
    image = cv2.resize(image, (image_shape[1], image_shape[0]), interpolation=cv2.INTER_AREA)
    #print('  Resized to', image.shape, 'in order to fit into the model')
    candidates.append(image)
    candidates_names.append(name)

# Show the candidates
%matplotlib inline
plt.rcParams["figure.figsize"] = [12, 15]
n_img_to_show = len(candidates)
num_cols = 4
num_rows = np.ceil(n_img_to_show / num_cols)
k = 1
for i in range(len(candidates)):
    image = candidates[i]
    subplot = plt.subplot(num_rows, num_cols, k)
    subplot.imshow(image)
    subplot.set_title(candidates_names[i])
    k = k + 1

print("\nShowing {0} candidates:".format(n_img_to_show))

# Preprocess the candidate images to prepare them for the prediction pipeline
candidates_processed = []
for i in range(len(candidates)):
    image = candidates[i]
    image_proccessed = preprocess_image(image)
    candidates_processed.append(image_proccessed)

### Question 6

_Choose five candidate images of traffic signs and provide them in the report. Are there any particular qualities of the image(s) that might make classification difficult? It would be helpful to plot the images in the notebook._



**Answer:**

I chose 10 images from the internet: some are similar to what the model has been trained with, others have some particularities that should pose a challenge to the model (90 km/h sign, vandalized sign, North American sign, French sign, old sign, speed limit sign with lights, etc).

Also, among the things that might have a negative impact on classification, is the fact that the preprocessing might not be totally appropriate. For it 'blindly' crops the images to the expected square ratio, but does not try to center the sign in the cropped result. Properly centering the sign in the preprocessed result should have a positive impact on the prediction accurracy. Future trainings with faked data (especially by applying translations), or better preprocessing should improve this aspect.

It would also be interesting to train the model with color images and balanced labels in the dataset. These are likely to improve its prediction accurracy on real images. 

In [4]:
### Run the predictions here.
### Feel free to use as many code cells as needed.

# For each candidate, calculate and print the predicted label
with tf.Session() as session:
    # Restore the persisted model
    tf.train.Saver().restore(session, model_to_use.path)
    print("Model '{0}' restored from '{1}'\n".format(model_to_use.name, model_to_use.path))
    
    predicted = session.run(tf.argmax(prediction, 1), feed_dict={features: candidates_processed, keep_prob_conv: 1, keep_prob: 1})
    for i in range(len(candidates)):
        print("Image {0:2d} '{1:15s}' has a predicted label of {2:2d}: {3}".format(i, candidates_names[i], predicted[i], label_names[predicted[i]][1]))

### Question 7

_Is your model able to perform equally well on captured pictures or a live camera stream when compared to testing on the dataset?_


**Answer:**

The model shows interesting results when encountering images different from its training data.
For example, the 90 km/h sign is predicted to be a 30 km/h sign which makes sense (other runs have preditected to be a 50 km/h sign). It is quite surprising to see the North American 'No Left Turn' predicted as 'Turn right ahead', but that could be due to the red cross on top of the sign.

The vandalized 'No Entry' sign is correctly predicted, as well as the old 'No passing' French sign.

The model has trouble with the Solar 50 km/h sign which is incorrectly labeled as 60 km/h.

In conclusion, the model does well with images close to the images it was trained with, even if it's not totally accurate. It also seems robust to vandalized or old signs.

In [ ]:
### Visualize the softmax probabilities here.
### Feel free to use as many code cells as needed.

# For each candidate, get and display the top 5 predicted labels with the corresponding confidence values
with tf.Session() as session:
    # Restore the persisted model
    tf.train.Saver().restore(session, model_to_use.path)
    print("Model '{0}' restored from '{1}'\n".format(model_to_use.name, model_to_use.path))
    
    top_5_values, top_5_indices = session.run(tf.nn.top_k(prediction, 5), feed_dict={features: candidates_processed, keep_prob_conv: 1, keep_prob: 1})
    for i in range(len(candidates)):
        print("Top 5 predictions for image {0} '{1}':".format(i, candidates_names[i]))
        for j in range(len(top_5_indices[i])):
            print('  {0:5.2f}%: Label {1:2d} - {2}'.format(top_5_values[i][j] * 100, top_5_indices[i][j], label_names[top_5_indices[i][j]][1]))

### Question 8

*Use the model's softmax probabilities to visualize the **certainty** of its predictions, [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.11/api_docs/python/nn.html#top_k) could prove helpful here. Which predictions is the model certain of? Uncertain? If the model was incorrect in its initial prediction, does the correct prediction appear in the top k? (k should be 5 at most)*


**Answer:**

The model seems to be pretty certain of its predictions, even the incorrect ones. In previous trainings, the model would give more mitigated probabilities and, if incorrect, would have most of the correct predictions in the top 5. This does not seem to be the case in this training run, which might be another sign of the overfitting of the training set.

### Question 9
_If necessary, provide documentation for how an interface was built for your model to load and classify newly-acquired images._


**Answer:**

I created a simple interface that loads new images, sets them to the same aspect ratio of the images that the model has been trained with, and then scales them to the size the model can process.

Afterwards, each image is preprocessed in the same fashion as the training data and then fed to the model to calculate the label that the model predicts for it.

In conclusion, this interface is well automated and can be easily augmented with a UI (web or desktop) for external users.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.